<a href="https://colab.research.google.com/github/mutatiismutandiis/master-projects/blob/main/01-EMDSV-HT2024-A3/EMDSV_A3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### EMDSV-HT24
### Assignment 3 - Group Y - Quantitative part

**Q6: How does the relationship between working during studies and learning look like?**


In [ ]:
# Mont Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Import libraries
import pandas as pd
import numpy as np
import re
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from scipy.stats import mannwhitneyu, spearmanr, chi2_contingency, fisher_exact
import statsmodels.api as sm # To perform Fisher's Exact test with larger tables

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Import data from the Excel file
route = '/content/drive/MyDrive/CURSOS Y BECAS/2024 - Master/EMDSV/Dataset-Survey-HT24.xlsx'

# Read the variable view (first sheet)
variables_view = pd.read_excel(route, sheet_name=0)
# Read the actual data (second sheet)
data = pd.read_excel(route, sheet_name=1)

# Display the first few rows of each for inspection
print(variables_view.head())
print(data.head())

  Variable Name                                              Label  \
0        Gender                                             Gender   
1           YoB                                      Year of birth   
2    Employment  Do you work at the same time while studying at...   
3    CommuteMin  What is your commuting distance to the campus ...   
4       Program  Which of DSV's education programs do you curre...   

                            Type Data Type  \
0  Single-Choice - Radio Buttons   Numeric   
1               Numeric - Slider   Numeric   
2  Single-Choice - Radio Buttons   Numeric   
3               Numeric - Slider   Numeric   
4  Single-Choice - Radio Buttons   Numeric   

                                         Value Codes  Missing Code  
0  1 = Man.\n2 = Woman.\n3 = Other/diverse.\n4 = ...         999.0  
1                                               none           NaN  
2  1 = Full time employment.\n2 = Part time emplo...         999.0  
3                         

In [ ]:
# Select variables needed for analysis
variables_of_interest = ['Employment', 'StudyHrs', 'MotivPass', 'SpprtVideo']
subset_data = data[variables_of_interest]

# Check the first few rows
print(subset_data.head())

# Check for missing values and data types
print(subset_data.isnull().sum())
print(subset_data.dtypes)


   Employment  StudyHrs  MotivPass  SpprtVideo
0           1         6          3           5
1           1         5          2           5
2           3         4          4           6
3           1         6          3           6
4           3         4          3           4
Employment    0
StudyHrs      0
MotivPass     0
SpprtVideo    0
dtype: int64
Employment    int64
StudyHrs      int64
MotivPass     int64
SpprtVideo    int64
dtype: object


**Notes:**

*  Selected variables:
  *  `Employment`: Coded as integers, representing categories:
      * 1 = Full time employment
      * 2 = Part time employment
      * 3 = I do not work while being a student at DSV.
  *  `StudyHrs`: Weekly study hours, coded as integers.
  *  `MotivPass`: Measures the agreement/disagreement of the afirmation: *'My main motivation is to pass that part rather than learn a lot or to get a high grade'*, coded as integers, representing the following categories:
      * 1 = Completely disagree
      * 2 = Mostly disagree
      * 3 = Somewhat disagree
      * 4 = Neither agree nor disagree
      * 5 = Somewhat agree
      * 6 = Mostly agree
      * 7 = Completely agree
  *  `SpprtVideo`: Measures the agreement/disagreement of the afirmation: *'I looked for support in the video lectures when I needed it'*, also coded as integers, representing the same categories that the previous one.
*  There are no missing values in the selected variables, so we don't need to handle them.
*  All variables are currently stored as integers (int64).

### General descriptive analysis



In [ ]:
# Create a copy to prevent issues
subset_data = data[variables_of_interest].copy()

# Convert variables to categorical
subset_data['Employment'] = subset_data['Employment'].astype('category')
subset_data['MotivPass'] = subset_data['MotivPass'].astype('category')
subset_data['SpprtVideo'] = subset_data['SpprtVideo'].astype('category')

# Verify the changes
print(subset_data.dtypes)

# Frequency counts for Employment
employment_counts = subset_data['Employment'].value_counts().sort_index()
print("Employment counts:\n", employment_counts)

# Add category labels for Employment
employment_labels = {
    1: "Full-time employment",
    2: "Part-time employment",
    3: "Not employed"
}

# Map labels to Employment counts
employment_counts.index = employment_counts.index.map(employment_labels)

# Pie chart for Employment
fig_employment = px.pie(values=employment_counts.values,
                        names=employment_counts.index,
                        title='Employment Status Distribution',
                        labels={'index': 'Employment Status', 'values': 'Count'})
fig_employment.show()

Employment    category
StudyHrs         int64
MotivPass     category
SpprtVideo    category
dtype: object
Employment counts:
 Employment
1    57
2    13
3     6
Name: count, dtype: int64


In [ ]:
# Frequency counts for MotivPass and SpprtVideo
motivpass_counts = subset_data['MotivPass'].value_counts().sort_index()
spprtvideo_counts = subset_data['SpprtVideo'].value_counts().sort_index()

print("MotivPass counts:\n", motivpass_counts)
print("SpprtVideo counts:\n", spprtvideo_counts)

# Bar chart for MotivPass
fig_motivpass = px.bar(x=motivpass_counts.index,
                       y=motivpass_counts.values,
                       title='Motivation to Pass Distribution',
                       labels={'x': 'Motivation Level', 'y': 'Count'})
fig_motivpass.show()

# Bar chart for SpprtVideo
fig_spprtvideo = px.bar(x=spprtvideo_counts.index,
                        y=spprtvideo_counts.values,
                        title='Support in Video Lectures Distribution',
                        labels={'x': 'Agreement Level', 'y': 'Count'})
fig_spprtvideo.show()


MotivPass counts:
 MotivPass
1    15
2    21
3    19
4     6
5     7
6     6
7     2
Name: count, dtype: int64
SpprtVideo counts:
 SpprtVideo
2     1
3     2
4     4
5    16
6    19
7    34
Name: count, dtype: int64


In [ ]:
# Check occurrences of StudyHrs categories 7 and 1
print("Counts for StudyHrs categories:\n", subset_data['StudyHrs'].value_counts().sort_index())

Counts for StudyHrs categories:
 StudyHrs
2     1
3     6
4    21
5    36
6    11
7     1
Name: count, dtype: int64


*Notes*:
* *There are no responses with category 1, we can discard it.*
* *There is only one student that chose category 7, so we will consider it as part of category 6 for the sake of visualization.*

In [ ]:
# Exclude categories 7 and 1
valid_studyhrs = subset_data[~subset_data['StudyHrs'].isin([7, 1])]

# Verify the remaining categories
print("Filtered StudyHrs counts:\n", valid_studyhrs['StudyHrs'].value_counts().sort_index())

# Map new intervals
study_hrs_intervals = {
    2: "(40, 50]",
    3: "(30, 40]",
    4: "(20, 30]",
    5: "(10, 20]",
    6: "[0, 10]",
    7: "[0, 10]" # Merge 0 hours with the following interval
}

# Create a new column with interval mappings
subset_data = subset_data.copy()  # Avoid SettingWithCopyWarning
subset_data['StudyHrsInterval'] = subset_data['StudyHrs'].astype(int).map(study_hrs_intervals)

# Verify the mapping
print(subset_data[['StudyHrs', 'StudyHrsInterval']].head())

# Histogram with adjusted appearance
fig_studyhrs = px.histogram(subset_data,
                            x='StudyHrsInterval',
                            category_orders={'StudyHrsInterval': ["[0, 10]", "(10, 20]", "(20, 30]", "(30, 40]", "(40, 50]"]},
                            title='Study Hours Distribution',
                            labels={'StudyHrsInterval': 'Study Hours Interval', 'count': 'Frequency'},
                            text_auto=True)  # Add counts on bars

# Adjust bar appearance
fig_studyhrs.update_traces(marker_line_width=1.5, marker_line_color="black")  # Add bar borders
fig_studyhrs.update_layout(bargap=0)  # Remove space between bars

# Update axes titles
fig_studyhrs.update_xaxes(title='Study Hours Interval')
fig_studyhrs.update_yaxes(title='Frequency')

# Show the histogram
fig_studyhrs.show()


Filtered StudyHrs counts:
 StudyHrs
2     1
3     6
4    21
5    36
6    11
Name: count, dtype: int64
   StudyHrs StudyHrsInterval
0         6          [0, 10]
1         5         (10, 20]
2         4         (20, 30]
3         6          [0, 10]
4         4         (20, 30]


In [ ]:
# Estimated descriptive statistics for "Study hours"
interval_midpoints = {
    "[0, 10]": 5,
    "(10, 20]": 15,
    "(20, 30]": 25,
    "(30, 40]": 35,
    "(40, 50]": 45
}

# Map the midpoints to the StudyHrsInterval column
subset_data['StudyHrsMidpoint'] = subset_data['StudyHrsInterval'].map(interval_midpoints)

# Calculate statistics
mean_studyhrs = subset_data['StudyHrsMidpoint'].mean()
std_studyhrs = subset_data['StudyHrsMidpoint'].std()
q1_studyhrs = subset_data['StudyHrsMidpoint'].quantile(0.25)
median_studyhrs = subset_data['StudyHrsMidpoint'].median()
q3_studyhrs = subset_data['StudyHrsMidpoint'].quantile(0.75)

# Print the results
print(f"Mean Study Hours: {mean_studyhrs}")
print(f"Standard Deviation: {std_studyhrs}")
print(f"Q1 Study Hours: {q1_studyhrs}")
print(f"Median Study Hours: {median_studyhrs}")
print(f"Q3 Study Hours: {q3_studyhrs}")


Mean Study Hours: 18.157894736842106
Standard Deviation: 8.825799501580876
Q1 Study Hours: 15.0
Median Study Hours: 15.0
Q3 Study Hours: 25.0


**Observations:**

* **Employment** (`Employment`):
  * 1: 57 students (majority; likely "full-time employment").
  * 2: 13 students (likely "part-time employment").
  * 3: 6 students (likely "unemployed"). <br>
  * **A large majority of students are full-time employees, which may affect the statistical analysis.**

* **Motivation to pass** (`MotivPass`): Most responses are low (1, 2, 3), indicating that most of the students disagree with the statement *'My main motivation is to pass that part rather than learn a lot or to get a high grade'*.

* **Video support** (`SpprtVideo`): Most responses are high (7, 6, 5), indicating positive experiences with video lectures. **This suggest strong agreement about the usefulness of video lectures**.

* **Study Hours** (`StudyHrs`):
  * In average, the students studied 18,15 hours per week.
  * 50% of the students study between 25-25 hours weekly.

### Q6.1: Working vs. non-working students, regarding the number of hours per week spent studying

In [ ]:
# Q6.1: Working vs not working and study hours -> Mann-Whitney U Test

# We group full-time and part-time workers as "working students"
subset_data['EmploymentStatus'] = np.where(subset_data['Employment'].isin([1, 2]), 1, 0)
# Check group sizes
print("Group sizes after transformation:")
print(subset_data['EmploymentStatus'].value_counts())

# Split study hours by group
working_hours = subset_data.loc[subset_data['EmploymentStatus'] == 1, 'StudyHrs']
non_working_hours = subset_data.loc[subset_data['EmploymentStatus'] == 0, 'StudyHrs']

# Perform the Mann-Whitney U Test
test_stat, p_value = mannwhitneyu(working_hours, non_working_hours, alternative='two-sided')
print("Mann-Whitney U Test Results:")
print(f"Test Statistic: {test_stat}")
print(f"P-value: {p_value}")

# Calculate descriptive statistics for each group
study_hrs_distribution = subset_data.groupby(['EmploymentStatus', 'StudyHrsInterval']).size().reset_index(name='Frequency')
# Calculate median interval for each group
median_intervals = subset_data.groupby('EmploymentStatus')['StudyHrs'].median().map(study_hrs_intervals)
# Display results
print("Distribution of Study Hours by Group:\n", study_hrs_distribution)
print("\nMedian Study Hours Interval by Group:\n", median_intervals)


Group sizes after transformation:
EmploymentStatus
1    70
0     6
Name: count, dtype: int64
Mann-Whitney U Test Results:
Test Statistic: 321.5
P-value: 0.02182692664051135
Distribution of Study Hours by Group:
    EmploymentStatus StudyHrsInterval  Frequency
0                 0         (10, 20]          1
1                 0         (20, 30]          4
2                 0         (40, 50]          1
3                 1         (10, 20]         35
4                 1         (20, 30]         17
5                 1         (30, 40]          6
6                 1          [0, 10]         12

Median Study Hours Interval by Group:
 EmploymentStatus
0    (20, 30]
1    (10, 20]
Name: StudyHrs, dtype: object


**Test results:**
* *Group Sizes*:
  * Working Students: 70 (very large group).
  * Non-Working Students: 6 (very small group).
  * The small size of the "Non-Working" group can make statistical inference less robust because tests like the Mann-Whitney U are sensitive to unbalanced group sizes.

* *Mann-Whitney U Test Results*:
  * U Statistic: 321.5
  * P-value: 0.0218 (significant at a 0.05 level).
  * Interpretation: Since the p-value is below 0.05, we reject the null hypothesis that there is no difference in study hours between working and non-working students. **This suggests there is a statistically significant difference in weekly study hours between these groups.**

* *Descriptive Statistics*:
  * Non-working students (0):
    * Most students (4 out of 6) fall in the (20, 30] interval.
    * There’s 1 student each in (10, 20] and (40, 50] intervals.
    * Median is (20, 30].
  * Working students (1):
    * Most students (35 out of 70) are in the (10, 20] interval.
    * Smaller groups are distributed across (20, 30], (30, 40], and [0, 10].
    * Median is (10, 20].
  * Non-working students tend to spend more time studying than working students, as their median study hours fall in a higher interval.

### Q6.2: Relation between weekly hours of study and motivation


In [ ]:
# Q6.2: Relation between Study hours and Motivation to Pass -> Spearman's Rank Correlation
# Ensure MotivPass is treated as ordinal
subset_data['MotivPass'] = subset_data['MotivPass'].astype(int)

# Calculate Spearman's correlation
correlation, p_value = spearmanr(subset_data['StudyHrsMidpoint'], subset_data['MotivPass'])

# Display results
print("Spearman's Rank Correlation Results:")
print(f"Correlation Coefficient: {correlation}")
print(f"P-value: {p_value}")


Spearman's Rank Correlation Results:
Correlation Coefficient: -0.13440001936732046
P-value: 0.24706226643922913


In [ ]:
# Simulated data (replace with your subset_data)
x = subset_data['StudyHrsMidpoint']
y = subset_data['MotivPass']

# Create the combined plot with jittered points and box plot
fig = go.Figure()

# Add jittered scatter plot
fig.add_trace(go.Box(
    x=x, y=y,
    boxpoints='all',  # Shows all points
    jitter=0.3,       # Add slight jitter to avoid overlap
    pointpos=-1.8,    # Spread the points slightly to the left
    marker=dict(color='blue', opacity=0.6),
    line=dict(color='black'),
    name='Study Hours vs Motivation'
))

# Update layout
fig.update_layout(
    title="Study Hours vs Motivation to Pass",
    xaxis_title="Study Hours (Midpoints)",
    yaxis_title="Motivation to Pass (1-7)",
    template='plotly_white'
)

fig.show()

**Test results:**
- Since we have p-value = 0.247 > 0.05, there is no statistically significant relationship between weekly study hours and motivation to pass. *We can't discard the null hypothesis*.
- We can see a trend that makes motivation more centered into lower values while in the middle intervals for the study hours. So for these groups the motivation is mostly not just to pass the course.

  ### Q6.3: Relation between employment status and seeking support in the video **recordings**


In [ ]:
# Q6.3: Employment status and Seeks support in video lectures relation -> We try Chi-Square Test
# Create a contingency table for Employment and Seeking Support
contingency_table = pd.crosstab(subset_data['Employment'], subset_data['SpprtVideo'])
print("Contingency Table:")
print(contingency_table)

Contingency Table:
SpprtVideo  2  3  4   5   6   7
Employment                     
1           1  2  1  14  12  27
2           0  0  2   2   4   5
3           0  0  1   0   3   2


In [ ]:
# Perform Chi-Square Test of Independence
chi2_stat, p_value, dof, expected = chi2_contingency(contingency_table)

# Check the assumptions of Chi-Square Test
print("\nExpected Frequencies:")
print(expected)

# Check for expected frequency violations
violations = (expected < 1).sum()  # Number of cells with expected frequency < 1
small_freq = (expected < 5).sum()  # Number of cells with expected frequency < 5
total_cells = expected.size        # Total number of cells
percent_small_freq = (small_freq / total_cells) * 100  # Percentage of small frequencies

print(f"\nAssumption Check:")
print(f"Cells with expected frequency < 1: {violations}")
print(f"Cells with expected frequency < 5: {small_freq} ({percent_small_freq:.2f}% of total cells)")

# Evaluate if the assumptions are violated
if violations > 0:
    print("\nChi-Square Test is NOT valid: At least one expected frequency is < 1.")
elif percent_small_freq > 20:
    print("\nChi-Square Test is NOT valid: More than 20% of cells have expected frequencies < 5.")
else:
    print("\nChi-Square Test assumptions are met. Proceeding with the test...")


Expected Frequencies:
[[ 0.75        1.5         3.         12.         14.25       25.5       ]
 [ 0.17105263  0.34210526  0.68421053  2.73684211  3.25        5.81578947]
 [ 0.07894737  0.15789474  0.31578947  1.26315789  1.5         2.68421053]]

Assumption Check:
Cells with expected frequency < 1: 7
Cells with expected frequency < 5: 14 (77.78% of total cells)

Chi-Square Test is NOT valid: At least one expected frequency is < 1.


In [ ]:
# Since the conditions for Chi-Square are not met, we try Fisher's exact test
# Create the original 3x7 contingency table
contingency_table_3x7 = pd.crosstab(subset_data['Employment'], subset_data['SpprtVideo'])

print("3x7 Contingency Table:")
print(contingency_table_3x7)

# Attempt Fisher's Exact Test
try:
    # Perform Fisher's Exact Test for larger tables
    fisher_p_value_3x7 = sm.stats.Table2x2.from_data(contingency_table_3x7.to_numpy()).test_nominal_association().pvalue
    print("\nFisher's Exact Test Results for 3x7 Table:")
    print(f"P-value: {fisher_p_value_3x7}")
except Exception as e:
    print("\nFisher's Exact Test could not be performed on the 3x7 table.")
    print(f"Error: {e}")

3x7 Contingency Table:
SpprtVideo  2  3  4   5   6   7
Employment                     
1           1  2  1  14  12  27
2           0  0  2   2   4   5
3           0  0  1   0   3   2

Fisher's Exact Test Results for 3x7 Table:
P-value: 0.8969074635734545


In [ ]:
# Collapse Seeks Support into 3 categories
def collapse_support(value):
    if value in [1, 2, 3]:
        return 'Low'
    elif value == 4:
        return 'Neutral'
    else:
        return 'High'

# Collapse Employment Status into 2 categories
def collapse_employment(value):
    if value in [1, 2]:  # Full-time or Part-time
        return 'Employed'
    else:
        return 'Not working'

# Create new columns for the collapsed variables
subset_data['CollapsedSupport'] = subset_data['SpprtVideo'].apply(collapse_support)
subset_data['CollapsedEmployment'] = subset_data['Employment'].apply(collapse_employment)

# Create the 2x3 contingency table
contingency_table_2x3 = pd.crosstab(subset_data['CollapsedEmployment'], subset_data['CollapsedSupport'])

print("\n2x3 Contingency Table:")
print(contingency_table_2x3)

# Perform Fisher's Exact Test for the 2x3 table
try:
    fisher_p_value_2x3 = sm.stats.Table2x2.from_data(contingency_table_2x3.to_numpy()).test_nominal_association().pvalue
    print("\nFisher's Exact Test Results for 2x3 Table:")
    print(f"P-value: {fisher_p_value_2x3}")
except Exception as e:
    print("\nFisher's Exact Test could not be performed on the 2x3 table.")
    print(f"Error: {e}")


2x3 Contingency Table:
CollapsedSupport     High  Low  Neutral
CollapsedEmployment                    
Employed               64    3        3
Not working             5    0        1

Fisher's Exact Test Results for 2x3 Table:
P-value: 0.9999806683148182


In [ ]:
# Further collapse the variables to create a 2x2 table

# Re-use collapsed employment variable
subset_data['Employment_2x2'] = subset_data['Employment'].apply(lambda x: 'Employed' if x in [1, 2] else 'Not working')

# Collapse support into High (5-7) and Low (1-4)
subset_data['Support_2x2'] = subset_data['SpprtVideo'].apply(lambda x: 'High Support' if x >= 5 else 'Low Support')

# Create the 2x2 contingency table
contingency_table_2x2 = pd.crosstab(subset_data['Employment_2x2'], subset_data['Support_2x2'])
print("\n2x2 Contingency Table:")
print(contingency_table_2x2)

# Perform Fisher's Exact Test on the 2x2 table
try:
    odds_ratio, p_value = fisher_exact(contingency_table_2x2.to_numpy(), alternative='two-sided')
    print("\nFisher's Exact Test Results for 2x2 Table:")
    print(f"Odds Ratio: {odds_ratio}")
    print(f"P-value: {p_value}")
except Exception as e:
    print("\nFisher's Exact Test could not be performed on the 2x2 table.")
    print(f"Error: {e}")



2x2 Contingency Table:
Support_2x2     High Support  Low Support
Employment_2x2                           
Employed                  64            6
Not working                5            1

Fisher's Exact Test Results for 2x2 Table:
Odds Ratio: 2.1333333333333333
P-value: 0.4516601718039617


**Test results:**
- The conditions for the Chi-Square were not met.
- Fisher's Exact test gave p > 0,05 for the 3 different contincengy tables we tried, grouping cathegories. Therefore, we cannot discard the null hypothesis.